
Alunos:
  * Arthur Breno dos Reis Paula - 12547382
  * Fernando - 8556609
  * Mateus Israel Silva - 11735042

# Desafio: **Porto Seguro's Safe Driver Prediction**

OBJETIVO:
 - Calcular a probabilidade de um seguro automotivo ser acionado

# Biliotecas importadas

In [1]:
## Bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Imputção de dados
from sklearn.impute import KNNImputer

## Decomposição
from sklearn.decomposition import PCA

## Pre processamento
from sklearn.preprocessing import StandardScaler

Importação dos dados

In [2]:
path_train = '/Users/administrador/Documents/Pessoal/repositorios/data/kaggle_porto_seguro_driver/train_no_missing.csv'
dados_brutos = pd.read_csv(path_train)

In [3]:
dados_brutos.shape

(541860, 55)

In [4]:
dados_brutos.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
1,0,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
2,0,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
3,0,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0


# Pré Processamento

## Normalização

In [5]:
data_cols = dados_brutos.iloc[:, 1:].columns

In [6]:
dados_brutos[data_cols] = pd.DataFrame(StandardScaler().fit_transform(dados_brutos[data_cols]), columns = data_cols)
#dados_brutos_teste = pd.DataFrame(StandardScaler().fit_transform(dados_brutos_teste), columns = dados_brutos_teste.columns)

In [7]:
dados_brutos[data_cols].describe().round(1)

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0,...,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0,541860.0
mean,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,...,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
min,-1.0,-0.5,-1.7,-0.8,-0.3,-0.8,-0.6,-0.4,-0.5,-0.0,...,-2.3,-1.2,-1.7,-2.7,-0.4,-1.3,-1.1,-0.6,-0.7,-0.4
25%,-1.0,-0.5,-0.9,-0.8,-0.3,-0.8,-0.6,-0.4,-0.5,-0.0,...,-0.6,-0.4,-0.5,-0.6,-0.4,-1.3,-1.1,-0.6,-0.7,-0.4
50%,-0.5,-0.5,-0.2,-0.8,-0.3,-0.8,-0.6,-0.4,-0.5,-0.0,...,-0.2,-0.4,0.1,-0.2,-0.4,0.8,0.9,-0.6,-0.7,-0.4
75%,0.5,1.0,0.6,1.2,-0.3,1.2,-0.6,-0.4,-0.5,-0.0,...,0.7,0.5,0.7,0.5,-0.4,0.8,0.9,1.6,1.4,-0.4
max,2.6,4.0,2.4,1.2,4.1,1.2,1.8,2.2,2.1,56.6,...,5.8,7.1,6.0,5.6,2.7,0.8,0.9,1.6,1.4,2.4


## Seleção de Variaveis

### Correlação entre colunas não target


In [8]:
matriz_corr = dados_brutos.corr()

In [9]:
print (matriz_corr)

                  target  ps_ind_01  ps_ind_02_cat  ps_ind_03  ps_ind_04_cat  \
target          1.000000   0.019375       0.003245   0.011853       0.010723   
ps_ind_01       0.019375   1.000000       0.001595   0.219946       0.071142   
ps_ind_02_cat   0.003245   0.001595       1.000000  -0.050165       0.149227   
ps_ind_03       0.011853   0.219946      -0.050165   1.000000      -0.096800   
ps_ind_04_cat   0.010723   0.071142       0.149227  -0.096800       1.000000   
ps_ind_05_cat   0.032366   0.010353      -0.005954   0.035458       0.012283   
ps_ind_06_bin  -0.032331  -0.281974      -0.129163  -0.045901      -0.300601   
ps_ind_07_bin   0.030763   0.008613       0.017547  -0.061215       0.311870   
ps_ind_08_bin   0.014524   0.187009       0.031577   0.012411       0.046990   
ps_ind_09_bin  -0.007170   0.164130       0.111864   0.112433      -0.011000   
ps_ind_10_bin   0.001714   0.002359      -0.001430   0.006270       0.002189   
ps_ind_11_bin   0.001887  -0.016917     

In [10]:
matriz_corr_filtro = matriz_corr[abs(matriz_corr) > 0.5]
np.fill_diagonal(matriz_corr_filtro.values, np.nan)
matriz_corr_filtro = matriz_corr_filtro.dropna(how='all', axis=0).dropna(how='all', axis=1)

In [11]:
print (matriz_corr_filtro)

               ps_ind_11_bin  ps_ind_12_bin  ps_ind_14  ps_ind_16_bin  \
ps_ind_11_bin            NaN            NaN   0.564551            NaN   
ps_ind_12_bin            NaN            NaN   0.892464            NaN   
ps_ind_14           0.564551       0.892464        NaN            NaN   
ps_ind_16_bin            NaN            NaN        NaN            NaN   
ps_ind_17_bin            NaN            NaN        NaN      -0.516789   
ps_ind_18_bin            NaN            NaN        NaN      -0.595706   
ps_car_04_cat            NaN            NaN        NaN            NaN   
ps_car_12                NaN            NaN        NaN            NaN   
ps_car_13                NaN            NaN        NaN            NaN   
ps_car_14                NaN            NaN        NaN            NaN   
ps_car_15                NaN            NaN        NaN            NaN   

               ps_ind_17_bin  ps_ind_18_bin  ps_car_04_cat  ps_car_12  \
ps_ind_11_bin            NaN            NaN       

Analizando a tabela será retirada as colunas: ps_ind_14, ps_ind_16_bin, ps_car_04_cat, ps_car_12, ps_car_13.

Dessa forma não irá restar nenhuma coluna com correlação maior que 0.5

In [12]:
dropped_col = []
dropped_col.append(matriz_corr_filtro.columns[2])
dropped_col.append(matriz_corr_filtro.columns[3])
dropped_col.append(matriz_corr_filtro.columns[6])
dropped_col.append(matriz_corr_filtro.columns[7])
dropped_col.append(matriz_corr_filtro.columns[8])

dropped_col


['ps_ind_14', 'ps_ind_16_bin', 'ps_car_04_cat', 'ps_car_12', 'ps_car_13']

In [13]:
dados_brutos = dados_brutos.drop(dropped_col, axis=1).copy()
#dados_brutos_teste = dados_brutos_teste.drop(dropped_col, axis=1).copy()

In [14]:
dados_brutos.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0.040920,0.970423,0.202705,1.179580,-0.309852,-0.813339,1.761695,-0.449636,-0.484315,...,1.525024,-0.367116,1.255548,0.168533,-0.373761,0.769778,0.896154,-0.634772,-0.732716,2.350777
1,0,-0.464032,-0.535344,0.948576,-0.847759,-0.309852,-0.813339,-0.567635,2.224022,-0.484315,...,-1.047270,-0.367116,-1.105011,0.532714,-0.373761,0.769778,0.896154,-0.634772,1.364785,-0.425391
2,0,1.555776,3.981958,1.694447,1.179580,-0.309852,-0.813339,-0.567635,2.224022,-0.484315,...,-0.618554,0.464238,2.435828,-0.195648,-0.373761,0.769778,0.896154,-0.634772,1.364785,-0.425391
3,0,-0.968984,-0.535344,-0.916102,-0.847759,-0.309852,1.229500,-0.567635,-0.449636,-0.484315,...,-1.475986,0.464238,0.665409,0.532714,-0.373761,-1.299076,-1.115879,-0.634772,-0.732716,-0.425391
4,0,-0.968984,0.970423,-1.661973,1.179580,-0.309852,1.229500,-0.567635,-0.449636,-0.484315,...,-1.047270,-0.367116,-1.105011,-1.652372,-0.373761,-1.299076,-1.115879,1.575369,1.364785,-0.425391


In [15]:
dados_brutos.to_csv('/Users/administrador/Documents/Pessoal/repositorios/data/kaggle_porto_seguro_driver/train_no_missing_scaled.csv', index=False)